In [33]:
import os 
from langchain_chroma import Chroma
from langchain_unstructured import UnstructuredLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM

In [2]:
file_path = '/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/HPE Files/HPE ProLiant ML350 Gen11-a50004308enw.pdf'
chroma_path = '/Users/keithatienza/Desktop/Academics/Emergent Consulting [HPE]/HPE LLM v2/DB'

In [5]:
loader = UnstructuredLoader(file_path)

In [7]:
docs = loader.load()

INFO: pikepdf C++ to Python logger bridge initialized


In [9]:
docs = filter_complex_metadata(docs)

In [13]:
!ollama pull mxbai-embed-large

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 819c2adf5ce6... 100% ▕████████████████▏ 669 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling b837481ff855... 100% ▕████████████████▏   16 B                         
pulling 38badd946f91... 100% ▕████████████████▏  408 B                         
verifying sha256 digest 
writing manifest 
success 


In [16]:
db = Chroma(persist_directory=chroma_path, embedding_function=OllamaEmbeddings(model="mxbai-embed-large"))

/var/folders/1g/drrbdl5d5tz58v9sn33gqfb40000gn/T/ipykernel_1711/2289304909.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = Chroma(persist_directory=chroma_path, embedding_function=OllamaEmbeddings(model="mxbai-embed-large"))
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [18]:
db.add_documents(docs)

['db2063f7-bc92-4dcc-b91b-a6a8587ba324',
 'd002f65a-b0f2-4972-9126-50d8a9d8c547',
 'd4b534ed-8099-4644-ba3b-c43f15c4eeba',
 '3a956edf-2c21-4978-9fdc-c660a0166615',
 '6abaf809-9a5f-4f5e-a88d-6460614dd680',
 '1bd1e9e7-2ca4-4dfe-9d7a-5626c32a2386',
 'fef1637d-928a-4276-962f-93f5ac85a88b',
 '44ed8acb-c416-4182-8255-e7ce9e7134b5',
 '68cb8ad2-f14f-4016-976f-e84e5a163276',
 '118ba76c-0bca-4614-ba65-bb333bed16d5',
 '571b4516-c7dd-4bb1-9b41-3901028c381f',
 '93c97690-7ac8-4eb5-8e82-7a32f0da4675',
 '85c17f58-4d18-4451-94e7-f4bae25e05a0',
 '5949a8a7-eb17-4096-a0d7-bb98ea397056',
 '7a9b339d-39cd-4bab-8083-78243cac66cb',
 '5e4dcac3-80ef-44b1-b0a1-ef67eae1e664',
 '09acb877-8129-4432-acc6-215740907495',
 'b4a4fb66-dd4a-475a-98af-4537a306478d',
 '81c09984-7378-44a8-9660-3d82c30e4d03',
 'bf6c64da-607f-460e-bb85-a616f89479fd',
 '99f135d1-e2ba-49fe-95be-38114522dcce',
 'f839ab32-561b-4f84-aed7-e905e0bac44b',
 '26000ab1-9bdc-438e-a35a-05b7b5dea2ee',
 '00cc87cc-d1d6-4399-a9a3-e58e47eba576',
 'd137fad2-0ed0-

In [23]:
prompt_template = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the context above: {question}
"""

In [27]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [45]:
def query_rag(query_text):
    results = db.similarity_search_with_score(query_text, k=10)
    context_text = "\n\n---\n\n".join([docs.page_content for docs, _score in results])
    prompt = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    model = OllamaLLM(model="llama3")
    response_text = model.invoke(prompt)
    formatted_response = f"Response: {response_text} \n\n"
    #print(formatted_response)
    return response_text

In [ ]:
answer = query_rag("Can you install RTX 4000 in this server?")

In [50]:
print(answer)

According to the notes for the NVIDIA RTX 4000 Ada Graphics Accelerator for HPE, it is supported by the HPE ProLiant ML350 Gen11 server. However, it requires a GPU Power Cable Kit (P47221-B21), which supports up to one accelerator.

Additionally, there are no specific restrictions or compatibility issues mentioned in the context that would prevent the installation of RTX 4000 in this server.
